<div style="text-align: center; padding-top: 30px; padding-bottom: 10px;">

<h1 style="font-size: 2.8em; font-weight: 600; margin-bottom: 0.2em;">
Global Neural Network Model
</h1>

<p style="font-size: 1.2em; color: gray; font-style: italic; margin-top: 0;">
This notebook visualises the main results from the paper.
</p>

</div>


##  1. Loading Packages and Data

In this section, we import required libraries, define the model parameters and load the dataset.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import os
from models.global_model.model_functions.helper_functions.prepare_data import Prepare
from utils import create_pred_input
from scipy.interpolate import griddata
from models import MultivariateModelGlobal as Model       


os.environ['PYTHONHASHSEED'] = str(0)

#model parameters                    
lr = 0.001                      # Learning rate
min_delta = 1e-6               # Tolerance for optimization
patience = 50                   # Patience for early stopping
verbose = 2                     # Verbosity mode for optimization
n_countries=196
time_periods=63                 #

#prepare the data
data=pd.read_excel('../data/MainData.xlsx')
growth, precip, temp = Prepare(data, n_countries, time_periods)
x_train = {0:temp, 1:precip}

#summary statics for standardisation
mean_temp=np.nanmean(data["TempPopWeight"])
std_temp=np.nanstd(data["TempPopWeight"])
mean_precip=np.nanmean(data["PrecipPopWeight"])
std_precip=np.nanstd(data["PrecipPopWeight"])

pred_input, T, P= create_pred_input(mc=False, mean_T=mean_temp, std_T=std_temp, mean_P=mean_precip, std_P=std_precip)




2025-11-13 13:33:33.565831: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-13 13:33:33.707708: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-13 13:33:33.715974: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-13 13:33:33.733735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763040813.766740   64332 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763040813.77

In [10]:
#load results from simulations and retrieve best n models
n_models = 10 
date_of_run = '2025-11-12'
Model_selection='IC'

results = dict(np.load(f'../results/metrics/{Model_selection}/{date_of_run}/results.npy', 
                       allow_pickle=True).item())

results={k: v for k,v in results.items() if v is not None}
top_models = sorted(results, key=lambda node: results[node][2])[0:n_models]

#print the corresponding BIC for all  the top models

print(f'Top models {top_models}, Holdout: {[results[model][2] for model in top_models]}')



Top models [(32, 2), (16, 2), (8, 2), (2, 2), (4, 4), (16, 2, 2), (4, 2), (2,), (4, 4, 4), (2, 2, 2)], Holdout: [np.float64(0.004287374671548605), np.float64(0.0042893365025520325), np.float64(0.004292081110179424), np.float64(0.00429602712392807), np.float64(0.004297249484807253), np.float64(0.0042993719689548016), np.float64(0.004301048349589109), np.float64(0.004303024150431156), np.float64(0.004307144787162542), np.float64(0.004307904280722141)]


# 2. Data analysis

## 2.1 Calculating the best ten surfaces, the average of those and the benchmark surface 


In [11]:

# --- Build surfaces for each of the top-n models ---------------------
model_surfaces = []
for idx, node in enumerate(top_models[0:n_models], 1):
    
    # instantiate and load your model
    factory = Model(node, x_train, growth, dropout=0, country_trends=False, dynamic_model=False, within_transform=False, add_fe=False)
    factory.Depth=len(node)
    model=factory.get_model()
    weight_file = f'../results/Model Parameters/{Model_selection}/{date_of_run}/{node}.weights.h5'
    # weight_file = f'../results/Model Parameters/IC/2025-05-11/(32, 2).weights.h5'
    
   
    model.load_params(weight_file)

    # # fit & predict
    # model.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=verbose)

    pred_flat = model.model_visual.predict([pred_input]).reshape(-1,)
    Growth = pred_flat.reshape(T.shape)

    opacity = 0.3
    surf = go.Surface(
        x=T, y=P/1000, z=Growth, #ensure that the surfaces are meassured in meters instead of milimeters
        colorscale='Cividis',
        opacity=0.85,
        showscale=False,
        name=f'Model {node}'
    )
    model_surfaces.append(surf)
 

#calculate the average surface
z=np.mean([surf.z for surf in model_surfaces], axis=0).reshape(T.shape)

mean_surface = go.Surface(
        x=T, y=P/1000, z=z,
        colorscale='Cividis',
        opacity=0.85,
        showscale=False,
        name='mean_surface'
    )


# --- Load benchmark data and create grid  ---------------------

benchmark_data = pd.read_csv('../data/Benchmark/3d_results.csv')
bench_temp   = benchmark_data['temp'].values
bench_precip = benchmark_data['precip_value'].values * 1000
bench_growth = benchmark_data['avg_prediction'].values

bench_Z = griddata(
    points=(bench_temp, bench_precip),
    values=bench_growth,
    xi=(T, P),
    method='linear'
)

# --- Create the benchmark surface --------------------------------------
bench_surface = go.Surface(
    x=T, y=P/1000, z=bench_Z,
    colorscale='Cividis',
    showscale=False,
    name='Benchmark'
)





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step


In [12]:
benchmark_data = pd.read_csv('../data/Benchmark/3d_results_no_outliers.csv')
bench_temp   = benchmark_data['temp'].values
bench_precip = benchmark_data['precip_value'].values * 1000
bench_growth = benchmark_data['avg_prediction'].values

bench_Z = griddata(
    points=(bench_temp, bench_precip),
    values=bench_growth,
    xi=(T, P),
    method='linear'
)

# --- Create the benchmark surface --------------------------------------
bench_surface_no_outlier = go.Surface(
    x=T, y=P/1000, z=bench_Z,
    colorscale='Cividis',
    showscale=False,
    name='Benchmark'
)


In [13]:
#add the true data as grid points / remember to undo the standardisation
true_data=go.Scatter3d(
  x=np.array(data["TempPopWeight"]).flatten(),
  y=np.array(data["PrecipPopWeight"]).flatten()/1000,
  z=np.array(data["GrowthWDI"]).flatten(),
  mode='markers',
  marker=dict(size=1.5, opacity=0.2, color='red')
  )
    

### 2.1.2 Data visualisation
In this section we visualise the top-n models in a temperature-precipitation grid 


In [14]:
n_obs_holdout=5*n_countries


holdout_data=go.Scatter3d(
  x=true_data.x[-n_obs_holdout:],
  y=true_data.y[-n_obs_holdout:],
  z=true_data.z[-n_obs_holdout:],
  mode='markers',
  marker=dict(size=1.5, opacity=0.5, color='blue'),
  name='Holdout Data'
  )

# plot_data = model_surfaces
# plot_data = [mean_surface] 
# plot_data=[mean_surface, bench_surface]
# plot_data=[bench_surface, bench_surface_no_outlier]
plot_data= model_surfaces + [holdout_data]


fig= go.Figure(data=plot_data)
fig.update_layout(
    scene=dict(
        xaxis_title='Temperature (°C)',
        yaxis_title='Precipitation (m)',
        zaxis_title='Δ ln(Growth)',
        camera=dict(eye=dict(x=2.11, y=0.12, z=0.38))
        
        ,zaxis=dict(range=[-0.5, 0.5])
    ),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

fig.show()



In [ ]:

fig.write_html("../results/images/holdout_mean_surface_(32,2).html")



## In this section I compare the in-sample fit of our Neural Network and a quadratic benchmark

In [ ]:
node=(2,)    
# instantiate and load your model
factory = Model(node, x_train, growth, dropout=0.2, penalty=0, country_trends=False)
factory.Depth=len(node)
model=factory.get_model()
weight_file = f'../results/Model Parameters/BIC/{node}.weights.h5'
model.load_params(weight_file)

# fit & predict
model.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=verbose)

pred_flat = model.model_visual.predict(pred_input).reshape(-1,)
Growth = pred_flat.reshape(T.shape) 

opacity = 0.3
surf = go.Surface(
    x=T, y=P/1000, z=Growth, #ensure that the surfaces are meassured in meters instead of milimeters
    colorscale='Cividis',
    opacity=0.85,
    showscale=False,
    name=f'Model {node}'
)

fig = go.Figure(data=surf)
fig.update_layout(
    scene=dict(
        xaxis_title='Temperature (°C)',
        yaxis_title='Precipitation (m)',
        zaxis_title='Δ ln(Growth)',
        camera=dict(eye=dict(x=2.11, y=0.12, z=0.38)),
        zaxis=dict(range=[-0.15, 0.15])
    ),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

fig.update_layout(
               autosize=False,
        width=500,
        height=600,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4,
        ),

            scene=dict(
                xaxis_title='Temperature (°C)',
                yaxis_title='Precipitation (m)',
                zaxis_title='Δ ln(Growth)',
                camera=dict(eye=dict(x=2.35, y=0.006, z=0.4))
                
            ),
            
            legend=dict(
                bgcolor='rgba(255,255,255,0.7)',
                bordercolor='black',
                borderwidth=1
            ),
            font=dict(
            size=10
        )
        )
fig.show()




### Dynamic model 


In [ ]:
node=(8,2,2)
date_of_run = '2025-09-22'
Model_selection='IC'

#fit the model 
# instantiate and load your model
factory = Model(node, x_train, growth, dropout=0, penalty=0, country_trends=False, dynamic_model=True)
factory.Depth=len(node)
model=factory.get_model()
weight_file = f'../results/Model Parameters/{Model_selection}/{date_of_run}/{node}.weights.h5'
model.load_params(weight_file)

# fit & predict
model.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=verbose)




In [ ]:

pred_input, T, P= create_pred_input(mc=False, mean_T=mean_temp, std_T=std_temp, mean_P=mean_precip, std_P=std_precip, time_periods=63)

print(pred_input)
pred_flat = model.model_visual.predict(pred_input).reshape(-1,)

Growth = pred_flat.reshape(T.shape) 



In [ ]:
# do the same for the real data, divide into groups by year 
years = data['Year'].unique()

#divide the data into groups by year
temp_by_year = [data[data['Year'] == year]['TempPopWeight'].values for year in years]
precip_by_year = [data[data['Year'] == year]['PrecipPopWeight'].values for year in years]
growth_by_year = [data[data['Year'] == year]['GrowthWDI'].values for year in years]




In [ ]:



model_array= np.array(Growth).ravel().reshape(90,90,64)

years = np.arange(1960, 1960 + model_array.shape[2]-1)  # or whatever years you want

temp= T[:, :, 0]       # 2D array of temperatures (shape: (n_precip, n_temp))
precip= P[:, :, 0]  

frames = []
for i, yr in enumerate(years):   # loop over time frames
   
    model_surf = go.Surface(
        x=temp, y=precip, z=model_array[:, :, i],
        name='Model', showscale=False, opacity=0.9, hoverinfo='skip'
    )
    
    x_obs = temp_by_year[i] 
    y_obs = precip_by_year[i]   # convert to meters
    z_obs = growth_by_year[i]


    obs_scatter = go.Scatter3d(
        x=x_obs, y=y_obs, z=z_obs,
        mode='markers',
        marker=dict(size=4, symbol='circle', line=dict(width=0.3), opacity=0.9),
        name='Observations',
        hoverinfo='text',
        showlegend=False
    )

    frames.append(go.Frame(data=[model_surf, obs_scatter], name=str(yr)))

    
# compute z-limits from model array (use a small margin if desired)
zmin = float(np.nanmin(model_array))
zmax = float(np.nanmax(model_array))
zmargin = 0.02 * (zmax - zmin) if (zmax - zmin) != 0 else 0.1
zmin -= zmargin
zmax += zmargin

# initial trace = first frame's model surface (frames[0].data[0] because each frame stores model_surf)
initial_surface = frames[0].data[0]
initial_obs = frames[0].data[1]

fig = go.Figure(data=[initial_surface, initial_obs], frames=frames)

# slider & play/pause (same idea as you had)
steps = []
for fr in frames:
    steps.append(dict(
        method="animate",
        args=[[fr.name],
              dict(mode="immediate",
                   frame=dict(duration=200, redraw=True),
                   transition=dict(duration=0))],
        label=fr.name
    ))
sliders = [dict(active=0, pad={"t": 50}, steps=steps, currentvalue={"prefix": "Year: "})]

common_camera = dict(eye=dict(x=2.11, y=0.12, z=0.38))

fig.update_layout(
    scene=dict(
        xaxis_title='Temperature (°C)',
        yaxis_title='Precipitation (m)',
        zaxis_title='Δ ln(Growth)',    # adjust label if your obs are on a different scale
        camera=common_camera,
        zaxis=dict(range=[zmin, zmax])
    ),
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        y=0.05,
        x=-0.01,
        xanchor="right",
        yanchor="top",
        pad={"t": 60, "r": 10},
        buttons=[
            dict(label="Play",
                 method="animate",
                 args=[None,
                       dict(frame=dict(duration=150, redraw=True),
                            transition=dict(duration=0),
                            fromcurrent=True,
                            mode='immediate')]),
            dict(label="Pause",
                 method="animate",
                 args=[[None],
                       dict(frame=dict(duration=0, redraw=False),
                            mode='immediate',
                            transition=dict(duration=0))])
        ],
    )],
    sliders=sliders,
    showlegend=False,
    height=600,
    width=900
)

fig.show()


In [ ]:
fig.write_html("../results/images/(8,2,2).html")

# Make 2d slice plots 

In [ ]:
import numpy as np
import plotly.graph_objects as go


# --- settings ---
percentile = 0                
selected_frame_indices = [1, 20, 40, 60]  # indices in the time axis 

# --- prepare 1D axis vectors 
precip_vals = np.unique(precip[:, 0])   # length = n_precip
temp_vals   = np.unique(temp[0, :])     # length = n_temp

# compute numeric target precipitation value for the requested percentile
target_precip = np.percentile(precip_vals, percentile)

# helper: for a given time-frame index, interpolate z across precip -> evaluate at target_precip for each temperature column
def cross_section_at_precip(frame_idx):
    n_temp = model_array.shape[1]
    z_at_temp = np.empty(n_temp, dtype=float)
    for j in range(n_temp):                       # for each temperature column
        z_profile = model_array[:, j, frame_idx]  # values along precip axis
        # np.interp requires x to be increasing: percip_vals should be sorted by np.unique
        z_at_temp[j] = np.interp(target_precip, precip_vals, z_profile)
    return z_at_temp

# --- build Plotly figure ---
fig2 = go.Figure()

for fi in selected_frame_indices:
    if fi < 0 or fi >= model_array.shape[2]:
        raise IndexError(f"Frame index {fi} is out of range (0..{model_array.shape[2]-1})")
    zcurve = cross_section_at_precip(fi)   # length n_temp
    label = f"idx {fi}"
    # if you have the 'years' array and want calendar years in legend:
    try:
        label = f"{years[fi]}"
    except Exception:
        pass
    fig2.add_trace(go.Scatter(x=temp_vals, y=zcurve, mode='lines', name=label))

fig2.update_layout(
    title=f"Cross-section at {percentile}th percentile of precipitation ≈ {target_precip:.4g}",
    xaxis_title="Temperature (°C)",
    yaxis_title="Δ ln(Growth)",
    legend_title="Year",
    height=500, width=800
)

fig2.show()


In [ ]:
fig2.write_image("../results/images/Paper/cross_section_precip0_(8,2,2).pdf", width=1600, height=1200, scale=2)

2.3 Marginal effects

In [ ]:
   ##Marginal effects are calculated by keeping precipitation at a constant, and then calculating the corresponding growth moving one temperature up. 

   # flatten the precip grid and compute 33rd percentile
   fixed_array = np.percentile(np.array(P.flatten()), q=99)

   # find the row index in P closest to that precip
   # (assuming P is shape (n_precip, n_temp) from meshgrid)
   idx = np.argmin(np.abs(P[:,0] - fixed_array))

   axis = T[idx, :]


   # extract model and benchmark slices
   model_slice = Growth[idx, :]
   bench_growth = bench_Z[idx, :]


   marginal_effects_NN = np.diff(model_slice)  # first difference to approximate marginal effects
   marginal_effects_Leirvik = np.diff(bench_growth)  # first difference to approximate marginal effects

   fig2d = go.Figure()



   # --- 2) make the 2D line plot --------------------------------------------
   fig2d.add_trace(go.Scatter(
            x=axis,
            y=marginal_effects_NN,
            mode='lines',
            line=dict(width=2)
      ))

   fig2d.add_trace(go.Scatter(
            x=axis,
            y=marginal_effects_Leirvik,
            mode='lines',
            line=dict(width=2)
      ))



   fig2d.update_layout(
   xaxis_title='Temperature (°C)',
   yaxis_title='Growth',
   xaxis=dict(range=[0, 30]),
   legend=dict(
      bgcolor='rgba(255,255,255,0.7)',
      bordercolor='black',
      borderwidth=1
   )
   )

   fig2d.show()





   

## 2.2 Comparing Fixed Effects

In [ ]:
#model parameters                    
lr = 0.001                      # Learning rate
min_delta = 1e-6               # Tolerance for optimization
patience = 50                   # Patience for early stopping
verbose = 2                     # Verbosity mode for optimization
n_countries=196
time_periods=63                 #

#define model of interest
node=(2,)
factory = Model(node, x_train, growth, dropout=0.2, penalty=0, country_trends=True)
factory.Depth=len(node)
model=factory.get_model()

weight_file = f'../results/Model Parameters/BIC/{node}.weights.h5'
model.load_params(weight_file)

# fit & predict
model.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=verbose)

model.alpha
theta_linear = model.linear_trend_layer.get_weights()[0].reshape(-1)  # length = number of countries included
theta_quad   = model.quadratic_trend_layer.get_weights()[0].reshape(-1)



In [ ]:

#compare country Fixed effects from benchmark and model 

bench_country_FE=np.load('../Benchmark/country_FE.npy', allow_pickle=True)


iso = np.asarray(bench_country_FE)[:,0]
arr = np.asarray(bench_country_FE)[:,1]
alpha_1d = np.asarray(model.alpha).squeeze()   # collapses (1,N) -> (N,)

print(arr.shape, alpha_1d.shape)

df=pd.DataFrame({
    'iso': iso,
    'bench_country_FE': arr,
    'model_alpha': alpha_1d
    ,'diff': arr - alpha_1d
})

print(df)

In [ ]:
#compare country Fixed effects from benchmark and model 

bench_time_FE=np.load('../Benchmark/time_FE.npy', allow_pickle=True)



iso = np.asarray(bench_time_FE)[:,0]
arr = np.asarray(bench_time_FE)[:,1]
alpha_1d = np.asarray(model.beta).squeeze()   # collapses (1,N) -> (N,)

print(arr.shape, alpha_1d.shape)

df=pd.DataFrame({
    'iso': iso,
    'bench_time_FE': arr,
    'model_alpha': alpha_1d
    ,'diff': arr - alpha_1d
})

print(df)

In [ ]:
#compare country Fixed effects from benchmark and model 

bench_linear_trend_FE=np.load('../Benchmark/linear_time_trend.npy', allow_pickle=True)



iso = np.asarray(bench_linear_trend_FE)[:,0]
arr = np.asarray(bench_linear_trend_FE)[:,1]
alpha_1d = np.asarray(theta_linear).squeeze()   # collapses (1,N) -> (N,)

print(arr.shape, alpha_1d.shape)

df=pd.DataFrame({
    'iso': iso,
    'bench_linear_trend': arr,
    'model_linear_trend': alpha_1d
    ,'diff': arr - alpha_1d
})

print(df)

In [ ]:
#time fixed effects benchmark data
bench_time=pd.read_csv('../data/Benchmark/time_fixed_effects_Burke.csv')

# cleaning up the benchmark time data
bench_time['time'] = bench_time['Unnamed: 0'].astype(str).str.extract(r'(\d{4})')
bench_time['time'] = bench_time['time'].astype(float).astype('Int64')  # nullable integer
bench_time.drop(columns=['Unnamed: 0'], inplace=True)


#comparing model time fixed effects with benchmark time fixed effects
plt.figure(figsize=(10, 6))
plt.plot(bench_time.iloc[:, 1], bench_time.iloc[:, 0], label='Benchmark', color='red')
plt.plot(model.beta, label='Model', color='blue')




## 2.3 Making 2-D plots

In [ ]:



# --- load benchmark and model predictions as before ---------------------
benchmark_data = pd.read_csv('../data/Benchmark/3d_results_Leirvik.csv')
bench_temp   = benchmark_data['temp'].values
bench_precip = benchmark_data['precip_value'].values * 1000
bench_growth = benchmark_data['avg_prediction'].values


def plot_two(percentiles, fixed_value, var_name):
    fig2d = go.Figure()

    for percentile in percentiles:
        
        # flatten the precip grid and compute 33rd percentile
        fixed_array = np.percentile(np.array(fixed_value.flatten()), q=percentile)

        # find the row index in P closest to that precip
        # (assuming P is shape (n_precip, n_temp) from meshgrid)
        idx = np.argmin(np.abs(fixed_value[:,0] - fixed_array))

        # extract the corresponding temperature axis
        if var_name == 'temp':
            axis = P[idx, :]
        else: 
            axis = T[idx, :]


        # extract model and benchmark slices
        model_slice = Growth[idx, :]
        bench_slice = bench_Z[idx, :]

        # --- 2) make the 2D line plot --------------------------------------------
        fig2d.add_trace(go.Scatter(
                x=axis,
                y=model_slice,
                mode='lines',
                name=f'{var_name} {percentile}th Percentile',
                line=dict(width=2)
            ))

    
    fig2d.update_layout(
        xaxis_title='Temperature (°C)',
        yaxis_title='Growth',
        xaxis=dict(range=[0, 30]),
        legend=dict(
            bgcolor='rgba(255,255,255,0.7)',
            bordercolor='black',
            borderwidth=1
        )
    )

    fig2d.show()




percentiles= range(0, 101, 33)  # 0th, 33rd, 66th, and 100th percentiles
# --- 3) (optional) save as html ------------------------------------------

# pio.write_html(fig2d, f'../results/images/2D_slice_Leirvik_comparison_{percentile}.html', auto_open=True)


## Appendix

In [ ]:



#visualise the model: 

print(model.alpha)

In [ ]:
np=model.beta.to_numpy()

last_siz=np[:58]
print('Last 6 sizes:', last_siz)

## 2d surfaces


In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px  # for a built‑in qualitative palette

# 1) compute the 90th‑percentile precip level
percentile = 50
p = np.percentile(data['PrecipPopWeight'], percentile)

# 2) tolerance window = 0.1 % of p
tol = 0.005 * p

# 3) slice all obs around that precip band
obs_slice = data[np.abs(data['PrecipPopWeight'] - p) <= tol].copy()

# 4) (no need for top‑3 any more)

# 5) base model trace
fig2d = go.Figure([
    go.Scatter(
        x=temp_axis,
        y=model_slice,
        mode='lines+markers',
        name=f'Model @ P≈{p:.1f} mm',
        line=dict(width=2, color='black'),
        marker=dict(size=6)
    )
])

# 6) pick a color palette and assign one color per country
countries = obs_slice['CountryName'].unique()
palette   = px.colors.qualitative.Plotly  # 10 distinct colors
color_map = {c: palette[i % len(palette)] for i,c in enumerate(countries)}

# 7) plot all observations, colored by country
for country in countries:
    df_ctry = obs_slice[obs_slice['CountryName'] == country]
    fig2d.add_trace(
        go.Scatter(
            x=df_ctry['TempPopWeight'],
            y=df_ctry['GrowthWDI'],
            mode='markers',
            name=country,
            marker=dict(
                size=8,
                color=color_map[country],
                symbol='circle'
            )
        )
    )

# 8) finalize layout
fig2d.update_layout(
    title=f'Growth vs Temperature at {percentile}th‑percentile Precipitation ({p:.1f} mm)',
    xaxis_title='Temperature (°C)',
    yaxis_title='Growth',
    xaxis=dict(range=[0, 30]),
    legend=dict(
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

fig2d.show()


## Model confidence plot - based on 10 best cv models

In [ ]:

results = dict(np.load(f'../results/metrics/15042025/results.npy', allow_pickle=True).item())

# Sort the keys (node configurations) by performance metric and select the top 10.
n_models = 10
date_of_run = '15042025'
ref_model = (8,2,2)  # Reference model configuration
top_models = sorted(results, key=lambda node: results[node])[:n_models]


# Initialize a list to store the prediction surfaces for each model.
prediction_surfaces = []
time_fixed_effects=[]
country_fixed_effects=[]

for node in top_models:
    
        # Instantiate your model with the given node configuration.
    model_instance = Model(nodes=node, x_train=x_train, y_train=growth, dropout=0, formulation="global")
    weight_file = f'../results/Model Parameters/CV/{date_of_run}/{str(node)}.weights.h5'
    model_instance.load_params(weight_file)  # Load the model weights
    model_instance.fit(lr=lr, min_delta=min_delta, patience=patience, verbose=2)
    
    model_instance.in_sample_predictions
    
    # Use the model to predict on the standardized (T,P) grid.
    growth_pred_flat = model_instance.model_visual.predict(pred_input)
    growth_pred_flat = np.reshape(growth_pred_flat, (-1,))   # shape: (900,)
    pred = growth_pred_flat.reshape(T.shape)  # reshape to (30, 30)
    
    prediction_surfaces.append(pred)
    time_fixed_effects.append(model_instance.beta)
    country_fixed_effects.append(model_instance.alpha)


# Convert the list into a NumPy array: shape (10, 30, 30)
prediction_surfaces = np.array(prediction_surfaces)

# Calculate the pointwise mean and standard deviation across the 10 models.
ensemble_mean = np.mean(prediction_surfaces, axis=0)
ensemble_std = np.std(prediction_surfaces, axis=0)

# Select the Chosen Model (e.g. configuration (8,2,2))
chosen_index = top_models.index(ref_model)
chosen_surface = prediction_surfaces[chosen_index]

model_confidence_plot(ref_model, chosen_surface, ensemble_std=ensemble_std, T=T, P=P, save_as_html=False)



## plotting the fixed effects from the model

In [ ]:

# Squeeze to shape (10, num_countries)
country_FE = np.array(country_fixed_effects).squeeze(axis=1)

# Number of models
n_models = country_FE.shape[0]

# Compute mean & sample std across the 10 models (result shape: num_countries)
country_FE_mean = np.mean(country_FE, axis=0).squeeze()
country_FE_std  = np.std(country_FE,  axis=0, ddof=1).squeeze()

# Standard error and 95% CI
se    = country_FE_std / np.sqrt(n_models)
t_val = stats.t.ppf(0.975, df=n_models - 1)
ci    = t_val * se


# X-axis indices 
country_indices = np.arange(country_FE_mean.shape[0])

plt.figure(figsize=(12, 6))
plt.plot(x, country_FE_mean, label='Mean')

plt.fill_between(
    country_indices,
    country_FE_mean - ci,
    country_FE_mean + ci,
    alpha=0.3,
    label='95% CI'
)
plt.title('Mean and 95% Confidence Interval of Country Fixed Effects')
plt.xlabel('Country Index')
plt.ylabel('Country Fixed Effect')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
